In [1]:
#解析 xml 之例外處理
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
#import 策略程式碼
import Ipynb_importer
import KD9
import Cont_3
import Gold_X_Death
import autoRemindPrice

importing Jupyter notebook from KD9.ipynb
importing Jupyter notebook from Cont_3.ipynb
importing Jupyter notebook from Gold_X_Death.ipynb
importing Jupyter notebook from autoRemindPrice.ipynb


In [3]:
# basic
import numpy as np
import pandas as pd
import requests
import re
import io
from bs4 import BeautifulSoup
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 
#talib
import talib
from talib import abstract
#gui
import tkinter as tk
from  tkinter import ttk  
from tkinter import messagebox 
import tkinter.font as tkFont

In [4]:
from tkinter.ttk import *

import matplotlib
matplotlib.use("TkAgg")

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [5]:
#爬蟲 yahoo csv
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"
    response = requests.post(url)
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [6]:
#爬取 api 並解析
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content)
    r = soup.find('string').text
    return r

## 第一種

In [99]:
#9日 K值低於 20% 買進，高於 80% 賣出
#9日 K值低於 20% 買進，高於 80% 賣出
def testKD92080set(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    #D = data.iloc[0:,2] #取 D 值
    #DIF = data2.iloc[0:,0] #取 DIF 值
    #DEA = data2.iloc[0:,1] #取 DEA 值
    #MACD = data2.iloc[0:,2] #取 MACD 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率
    datelist=[]
    kline =[]
    line20=[]
    line80=[]
    
    

    try:
        
        for i in range(len(K)):
            print(date[i])
            print(i)
            print(close[i])
            if(stock_count==0):
                if(float(start_money)>=close[i]*1000):
                    if(K[i]<20):
                        datelist.append(date[i])
                        kline.append(K[i])
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("尚未達到買點")
                        #continue
                else:
                    print("錢不購買")
                    #continue
            else:
                if(close[i] >= record[-1]+float(sProfit)): #現在的股價 >= 停利條件
                    datelist.append(date[i])
                    kline.append(K[i])
                    print('現在的股價=',close[i])
                    print('條件: 現股>=最後一次購買股價+10 = ') # recode[-1]為最後一次交易的股價
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價+5',record[-1]+float(sProfit))
                    print('已達停利標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(close[i] <= record[-1]-float(sLoss)):
                    datelist.append(date[i])
                    kline.append(K[i])
                    print('現在的股價=',close[i]) 
                    print('條件: 現股<=最後一次購買股價-5 = ')
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價-5',record[-1]-float(sLoss))
                    print('已達停損標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(K[i]<20):
                    if(float(start_money)>=close[i]*1000):
                        datelist.append(date[i])
                        kline.append(K[i])
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("錢不夠再買一張股票")
                        #continue
                elif(K[i]>80):
                    datelist.append(date[i])
                    kline.append(K[i])
                    print('K值為:',K[i])
                    print("手上有"+str(stock_count)+"張股票可賣")
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money
                else:
                    print("尚未達到買賣點")
                    #continue
        print("\n")          
        
        #textPanel.insert("end", rateOfReturn)
        textPanel1.insert("end", lastROP[-1])
        
        for i in range(len(datelist)):
            line20.append(20)
            line80.append(80)
        
        
        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)

        ax1.plot(datelist,kline,color= 'b',linewidth=1,linestyle="-",label='K')
        ax1.legend(loc = 'upper left', frameon=True)
        ax1.plot(datelist,line20,color= 'r',linewidth=1,linestyle="-")
        ax1.plot(datelist,line80,color= 'r',linewidth=1,linestyle="-")
        
        for tick in ax1.get_xticklabels():
            tick.set_rotation(33)
            tick.set_fontsize(6)


        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)  
        
        print('總報酬率',lastROP[-1])
        return lastROP[-1]
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第二種

In [102]:
#K值連續三天低於20後反彈(買進)，K值連續三天高於80後反彈(賣出)
def testKDcontSet(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    #bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率
    datelist=[]
    closelist=[]
    kline =[]
    line20=[]
    line80=[]
    print("2",start_money)
    try:
        for i in range(3,len(K)):
            if stock_count == 0:
                if float(start_money)>=close[i]*1000:
                    if K[i-1]<20 and K[i-2]<20 and K[i-3]<20: 
                        if K[i]>20: # K值連續 3 天小於 20 後上升
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            datelist.append(date[i-3])
                            datelist.append(date[i-2])
                            datelist.append(date[i-1])
                            datelist.append(date[i])
                            kline.append(K[i-3])
                            kline.append(K[i-2])
                            kline.append(K[i-1])
                            kline.append(K[i])
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                    else:
                        print("未達買點")
                        continue
                else:
                    print("資金不足")
                    continue

            else:
                if(close[i] >= record[-1]+float(sProfit)): #現在的股價 >= 停利條件
                        datelist.append(date[i-3])
                        datelist.append(date[i-2])
                        datelist.append(date[i-1])
                        datelist.append(date[i])
                        kline.append(K[i-3])
                        kline.append(K[i-2])
                        kline.append(K[i-1])
                        kline.append(K[i])
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股>=最後一次購買股價+5 = ') # recode[-1]為最後一次交易的股價
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價+5',record[-1]+float(sProfit))
                        print('已達停利標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue
                elif(close[i] <= record[-1]-float(sLoss)): #現在的股價 <= 停利條件
                        datelist.append(date[i-3])
                        datelist.append(date[i-2])
                        datelist.append(date[i-1])
                        datelist.append(date[i])
                        kline.append(K[i-3])
                        kline.append(K[i-2])
                        kline.append(K[i-1])
                        kline.append(K[i])
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股<=最後一次購買股價-5 = ')
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價-5',record[-1]-float(sLoss))
                        print('已達停損標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue

                elif K[i-1]<20 and K[i-2]<20 and K[i-3]<20:
                    if K[i]>20:
                        if float(start_money)>=close[i]*1000:
                            datelist.append(date[i-3])
                            datelist.append(date[i-2])
                            datelist.append(date[i-1])
                            datelist.append(date[i])
                            kline.append(K[i-3])
                            kline.append(K[i-2])
                            kline.append(K[i-1])
                            kline.append(K[i])
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        else:
                            print("資金不足")
                elif K[i-1]>80 and K[i-2]>80 and K[i-3]>80:
                    if K[i]<80:
                        datelist.append(date[i-3])
                        datelist.append(date[i-2])
                        datelist.append(date[i-1])
                        datelist.append(date[i])
                        kline.append(K[i-3])
                        kline.append(K[i-2])
                        kline.append(K[i-1])
                        kline.append(K[i])
                        print(date[i-1],'K值為:',K[i-1])
                        print(date[i-2],'K值為:',K[i-2])
                        print(date[i-3],'K值為:',K[i-3])
                        print(date[i],'K值為:',K[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:]=[]
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , )
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit))
                        #lastROP.append(rateOfReturn)
                        start_money = final_money
                else:
                    print(date[i])
                    print("未達買賣點")
        print("\n")        
        #textPanel.insert("end", lastROP[-1])
        textPanel2.insert("end", lastROP[-1])
        
        for i in range(len(datelist)):
            line20.append(20)
            line80.append(80)
        
        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)
        ax1.plot(datelist,kline,color= 'g',linewidth=1,marker='o',label='K')
        ax1.plot(datelist,line20,color= 'r',linewidth=0.5,linestyle="-")

        ax1.plot(datelist,line80,color= 'r',linewidth=0.5,linestyle="-")
        ax1.legend(loc = 'upper left', frameon=True)
        
        for tick in ax1.get_xticklabels():
            tick.set_rotation(33)
            tick.set_fontsize(6)

        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)

        print('總報酬率',lastROP[-1])
        return lastROP[-1] 
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第三種

In [9]:
## K值向上突破D值 (黃金交叉)，K值向下突破 D值 (死亡交叉)
def testKDcrossSet(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    D = data.iloc[0:,2] #取 D 值
    principal = float(start_money)
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    stock_count = 0 #股票張數
    bought_money = 0.0 #剩餘金額
    final_money = 0.0 #最終金額
    profit = 0.0 #獲利
    rateOfReturn = 0.0 #報酬率
    lastROP = []  
    record = []
    datelist = []
    kline=[]
    dline=[]
    try:  
        for i in range(1,len(K)):
            #print(date[i])
            print(date[i])
            print(i)
            print(close[i])
            datelist.append(date[i-1])
            kline.append(K[i-1])
            dline.append(D[i-1])
            if(len(record)==0): #手上沒股票
                if(float(start_money)>=close[i-1]*1000): #身上錢夠
                    if (K[i-1]<D[i-1] and K[i]>D[i]): #黃金交叉，買股票
                            stock_count = stock_count + 1
                            start_money = float(start_money) - (float(close[i-1])*1000)
                            record.append(close[i-1])
                            print("record:",record)
                            print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                            print("買了一張股票，股價為:",close[i-1])
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),start_money))
                    else: #不是黃金交叉，直接進入下個迴圈
                        continue
                else: #錢不夠，進入下個迴圈
                    continue
            elif(len(record)!=0): #身上有股票，判斷是否符合停利/停損，是->賣掉，否->判斷為黃金交叉(買)/死亡交叉(賣)/都不是(繼續)
                    if close[i-1] >= record[-1]+float(sProfit) :#停利，賣股票
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif close[i-1] <= record[-1]-float(sLoss) :
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif (K[i-1]<D[i-1] and K[i]>D[i]):#買
                        if float(start_money)>=close[i]*1000:
                            stock_count = stock_count + 1 #買一張股票
                            bought_money = start_money - (float(close[i-1])*1000) #剩餘資金 = 上一次交易剩餘資金 - 當日股價
                            record.append(close[i])
                            print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                            print("買了一張股票，股價為:",close[i-1])
                            print('剩餘資金為:',bought_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),bought_money))
                            start_money = bought_money
                        else:
                            continue
                    elif (K[i-1]>D[i-1] and K[i]<D[i]):
                        print(date[i],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        rateOfReturn = round(rateOfReturn, 2)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        lastROP.append(rateOfReturn)
                        print(lastROP)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money

        textPanel3.insert("end",lastROP[-1])

        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)
        ax1.plot(datelist,kline,color= 'b',linewidth=2,linestyle="-",label='K')
        ax1.legend(loc = 'upper left', frameon=True)
        ax2 = ax1.twinx()
        ax2.plot(datelist,dline,color= 'r',linewidth=2,linestyle="-",label='D')
        ax2.legend(loc = 'upper right', frameon=True)
        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)

        return lastROP[-1]
        
        
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

In [10]:
def first():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    data.reset_index(inplace=True)
    
    return  testKD92080set(start_money,close2,data,sProfit,sLoss)


In [11]:
def second():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    
    data.reset_index(inplace=True)
        
    return testKDcontSet(start_money,close2,data,sProfit,sLoss)


In [12]:
def third():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    #figure = Figure(figsize=(6, 3), dpi=100)
    #plot = figure.add_subplot(1, 1, 1)
    #plot.plot(data)
    #canvas = FigureCanvasTkAgg(figure, frm_r)
    #canvas.get_tk_widget().grid(row=1, column=0)
    
    data.reset_index(inplace=True)
    
    return testKDcrossSet(start_money,close2,data,sProfit,sLoss)

In [13]:
def callfunc2():
    ans = v.get()
    if(ans=="K值小於設定值進場，大於設定值出場"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        KD9.start(stockid,sProfit,sLoss)
    elif(ans=="K值連續3天小於設定值後反彈進場"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        Cont_3.start(stockid,sProfit,sLoss)
    elif(ans=="黃金交叉與死亡交叉"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        Gold_X_Death.start(stockid,sProfit,sLoss)
def callfunc():
    ans = v.get()
    if(ans=="K值小於設定值進場，大於設定值出場"):
        x = table.get_children()
        for item in x: 
            table.delete(item)
        #textPanel.delete('1.0','end') 
        textPanel1.delete('1.0','end') 
        textPanel2.delete('1.0','end') 
        textPanel3.delete('1.0','end')
        first()
    elif(ans=="K值連續3天小於設定值後反彈進場"):
        x = table.get_children()
        for item in x: 
            table.delete(item)
        #textPanel.delete('1.0','end') 
        textPanel1.delete('1.0','end') 
        textPanel2.delete('1.0','end') 
        textPanel3.delete('1.0','end')
        second()
    elif(ans=="黃金交叉與死亡交叉"):
        x = table.get_children()
        for item in x: 
            table.delete(item)
        #textPanel.delete('1.0','end') 
        textPanel1.delete('1.0','end') 
        textPanel2.delete('1.0','end') 
        textPanel3.delete('1.0','end')
        third()
def Run3():
    x = table.get_children()
    for item in x: 
        table.delete(item)
    #textPanel.delete('1.0','end') 
    textPanel1.delete('1.0','end') 
    textPanel2.delete('1.0','end') 
    textPanel3.delete('1.0','end')
    
    a = int(first())
    b = int(second())
    c = int(third())
    if a>b:
        if b>c:
            print("第一個策略獲利最多")
            bestanswer = "第一個策略獲利最多"
            tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
            first()
        else:
            if a>c:
                print('第一個策略獲利最多')
                bestanswer = "第一個策略獲利最多"
                tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
                first()

            else:
                print('第三個策略獲利最多')
                bestanswer = "第三個策略獲利最多"
                tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
                third()
    elif a<b:
        if b<c:
            print('第三個策略獲利最多')
            bestanswer = "第三個策略獲利最多"
            tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
            third()
        else:
            if c>a:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"
                tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
                second()
            else:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"
                tk.messagebox.showinfo(title='比較結果',  message=bestanswer)
                second()


In [24]:
def order():
    win.destroy()
    order = tk.Tk()
    order.title('下單')
    order.geometry("1100x600")
    frame = tk.Frame(order)
    frame.grid(row = 0)
    frame_l = tk.Frame(frame, ) #左視窗
    frame_r = tk.Frame(frame) #右視窗
    frame_c = tk.Frame(frame,)
    frame_l.grid(row = 1,column = 0)
    frame_r.grid(row = 1,column = 2)
    frame_c.grid(row = 1,column = 1)
    
    ft1 = tkFont.Font(family='Fixdsys', size=16, weight=tkFont.BOLD)
    
    
    
    
    def getOrderRecorde(): #委託紀錄id(

        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("委託時間","委託單ID","股票代碼","公司名稱","委託價格","委託數量","買賣類型","掛單類型")
        #表示列,不顯示
        table.column("委託時間",width=160)
        table.column("委託單ID",width=100)
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("委託價格",width=80)
        table.column("委託數量",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)

        #顯示標題
        table.heading("委託時間",text="委託時間")
        table.heading("委託單ID",text="委託單ID")
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("委託價格",text="委託價格")
        table.heading("委託數量",text="委託數量")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryWaitingOrderListGVE3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC"
        client = get_webservice(url)
        root2 = ET.fromstring(client)
        i = 1
        for country in root2.findall('Order'):
            print("第",i,"筆委託紀錄")
            OrderTime = country.get('OrderTime')
            OrderID = country.get('OrderID')
            AssetID = country.get('AssetID')
            CompName = country.get('CompName')
            Price = country.get('Price')
            Volume = country.get('Volume')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')

            print(OrderTime, OrderID, AssetID, CompName, Price, Volume, BSAction, OrderType)
            i+=1
            table.insert("","end",values=(OrderTime,OrderID,AssetID,CompName,Price,Volume,BSAction,OrderType))
            
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()

            
            
            
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getInstock(): #庫存
        table=ttk.Treeview(frame_r,show='headings') #表格
        table["columns"]=("股票代碼","公司名稱","持有數量","持有成本","現價","漲跌","盈虧","累積未實現損益")
        #表示列,不顯示
        table.column("股票代碼",width=160)
        table.column("公司名稱",width=80)
        table.column("持有數量",width=80)
        table.column("持有成本",width=80)
        table.column("現價",width=80)
        table.column("漲跌",width=80)
        table.column("盈虧",width=80)
        table.column("累積未實現損益",width=80)

        #顯示標題
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("持有數量",text="持有數量")
        table.heading("持有成本",text="持有成本")
        table.heading("現價",text="現價")
        table.heading("漲跌",text="漲跌")
        table.heading("盈虧",text="盈虧")
        table.heading("累積未實現損益",text="累積未實現損益")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVEAccount.asmx/QueryTodayPositionGve3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC&SubTotalItem=&SortItem=AssetCode%20Asc"
        client = get_webservice(url)
        root1 = ET.fromstring(client)
        i = 1
        for country in root1.findall('PortfolioAsset'):
            print("第",i,"筆庫存紀錄")
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            Hold = country.get('Hold')
            Cost = country.get('Cost')
            QuotePrice = country.get('QuotePrice')
            UpDown = country.get('UpDown')
            PL = country.get('PL')
            UnRealizedPL = country.get('UnRealizedPL')
            print(AssetCode, CompName, Hold, Cost, QuotePrice, UpDown, PL, UnRealizedPL)
            i+=1
            table.insert("","end",values=(AssetCode,CompName,Hold,Cost,QuotePrice,UpDown,PL,UnRealizedPL))
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getDeal(): #成交紀錄
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("交易時間","股票代碼","公司名稱","買賣類型","掛單類型","成交價","成交量","手續費","交易訊息")
        table.column("交易時間",width=160)   
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)
        table.column("成交價",width=80)
        table.column("成交量",width=80)
        table.column("手續費",width=80)
        table.column("交易訊息",width=80)

        table.heading("交易時間",text="交易時間")  
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        table.heading("成交價",text="成交價")
        table.heading("成交量",text="成交量")
        table.heading("手續費",text="手續費")
        table.heading("交易訊息",text="交易訊息")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryDealLogGVE3ByGMRDayRangeLiteXML_NS?GMRID=1111708496&StartDate=2019/01/02&EndDate=2019/12/20&Language=TC"
        client = get_webservice(url)
        root = ET.fromstring(client)
        i = 1
        for country in root.findall('GVEUserLog'):
            print("第",i,"筆成交紀錄")
            #LogID = country.get('LogID')
            LogTime = country.get('LogTime')
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')
            Price = country.get('Price')
            Volume = country.get('Volume')
            fee = country.get('fee')
            logdesc = country.get('logdesc')
            print(LogTime, AssetCode, CompName, BSAction, OrderType, Price, Volume, fee, logdesc)
            i+=1
            table.insert("","end",values=(LogTime,AssetCode,CompName,BSAction,OrderType,Price,Volume,fee,logdesc))
            
        def deletetable(): #清除
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getOrder(): #下單
    
            compcode = CompCode_Entry.get()
            price = Price_Entry.get()
            volume = Volume_Entry.get()
            bsaction = BSAction_Entry.get()
            ordertype = OrderType_Entry.get()

            url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/PutOrderXML3?GMRIDStr=BCSP&CompCode="+compcode+"&Price="+price+"&Volume="+volume+"&BSAction="+bsaction+"&OrderType="+ordertype+"&IsOddLot=0&Currency=TWD&OrderNote=ROD&OCType=0&CombineNo=&OrderParameter=0&Lang=TC&str_ip=127.0.0.1"
            client = get_webservice(url)
            new = client.split("\"")
            if new[1] == "Success":
                tk.messagebox.showinfo(title='回報', message='Success')
            elif new[1] == "Failure":
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:'+new[5])
            else:
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:格式輸入錯誤')

    def getPrice():
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("成交價","成交量","委買價","委買量","委賣價","委賣量","昨收價","漲停價","跌停價")
         #表示列,不顯示
        table.column("成交價",width=160)
        table.column("成交量",width=80)
        table.column("委買價",width=80)
        table.column("委買量",width=80)
        table.column("委賣價",width=80)
        table.column("委賣量",width=80)
        table.column("昨收價",width=80)
        table.column("漲停價",width=80)
        table.column("跌停價",width=80)
        #顯示標題
        table.heading("成交價",text="成交價")  
        table.heading("成交量",text="成交量")
        table.heading("委買價",text="委買價")
        table.heading("委買量",text="委買量")
        table.heading("委賣價",text="委賣價")
        table.heading("委賣量",text="委賣量")
        table.heading("昨收價",text="昨收價")
        table.heading("漲停價",text="漲停價")
        table.heading("跌停價",text="跌停價")
        table.grid(row = 0, column = 0)
        
        compcode = CompCode_Entry.get()
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryQuote5Price?compcode=" + compcode
        client = get_webservice(url)
        r = ET.fromstring(client)
        r_new = "您查詢的股價為:\n"+"成交價:"+str(r[0][0].text)+\
                                "\n"+"成交量:"+str(r[0][1].text)+\
                                "\n"+"委買價:"+str(r[0][2].text)+\
                                "\n"+"委買量:"+str(r[0][22].text)+\
                                "\n"+"委賣價:"+str(r[0][12].text)+\
                                "\n"+"委賣量:"+str(r[0][23].text)+\
                                "\n"+"昨收價:"+str(r[0][24].text)+\
                                "\n"+"漲停價:"+str(r[0][25].text)+\
                                "\n"+"跌停價:"+str(r[0][26].text)+"\n"
        
        
        table.insert("","end", values=(str(r[0][0].text),str(r[0][1].text),str(r[0][2].text),str(r[0][22].text),str(r[0][12].text),str(r[0][23].text),str(r[0][24].text),str(r[0][25].text),str(r[0][26].text)))
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
        
    def cancelOrder():
        cancle_id = cancle_entry.get()
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/CancelOrderStr_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&OrderID=" + cancle_id
        client = get_webservice(url)
        getOrderRecorde()

    #Label
    title = tk.Label(frame_l, text = "委託下單",font=ft1)
    title.grid(row=0,column=0)
    title = tk.Label(frame_l, text = "查詢功能",font=ft1)
    title.grid(row=7,column=0)
    CompCode = tk.Label(frame_l, text="股票代碼")#股票代碼
    CompCode.grid(row=1,column=0)
    Price = tk.Label(frame_l, text="委託價格")#價格
    Price.grid(row=2,column=0)
    Volumn = tk.Label(frame_l, text="委託數量")#成交量
    Volumn.grid(row=3,column=0)
    BSAction = tk.Label(frame_l, text="買賣類型")#使用代碼 普通買進(B),普通賣出(S),融資買入(MB),融資賣出(RB),融券買入(RS)
    BSAction.grid(row=4,column=0)
    OrderType = tk.Label(frame_l, text="下單類型")#下單類型 漲停價(LMTU),跌停價(LMTD),限價單(LMT)，期權市價單：MKT
    OrderType.grid(row=5,column=0)
    cancleOrder = tk.Label(frame_l, text = "取消委託",font=ft1)#取消委託
    cancleOrder.grid(row=14,column=0)
    cancle = tk.Label(frame_l, text = "輸入ID:")
    cancle.grid(row = 15, column = 0)
    
    #預設輸入
    GMRIDstr = tk.StringVar(value="BCSP")
    CompCode = tk.StringVar(value="2330.tw")
    Volume = tk.StringVar(value="1000")
    BSAction = tk.StringVar(value="B")
    OrderType = tk.StringVar(value="MKT")

    #輸入框
    CompCode_Entry = tk.Entry(frame_l,textvariable=CompCode)
    CompCode_Entry.grid(row=1,column=1)
    Price_Entry = tk.Entry(frame_l,textvariable=Price)
    Price_Entry.grid(row=2,column=1)
    Volume_Entry = tk.Entry(frame_l,textvariable=Volume)
    Volume_Entry.grid(row=3,column=1)
    BSAction_Entry = tk.Entry(frame_l,textvariable=BSAction)
    BSAction_Entry.grid(row=4,column=1)
    OrderType_Entry = tk.Entry(frame_l,textvariable=OrderType)
    OrderType_Entry.grid(row=5,column=1)
    cancle_entry = tk.Entry(frame_l)
    cancle_entry.grid(row=15,column = 1)
#-------------------------Button-------------------------#    
    order_btn = tk.Button(frame_l , text="下單", command=getOrder) 
    order_btn.grid(row = 3,column = 2,padx = 10)    
    record_btn = tk.Button(frame_l, text="委託紀錄", command=getOrderRecorde)
    record_btn.grid(row = 8,column = 0,pady = 10)
    instock_btn = tk.Button(frame_l, text="庫存紀錄", command=getInstock)
    instock_btn.grid(row = 8,column = 1,pady = 10)
    deal_btn = tk.Button(frame_l, text="成交紀錄", command = getDeal)
    deal_btn.grid(row = 9,column = 0,pady = 10)
    stockprice_btn = tk.Button(frame_l, text="即時股價", command=getPrice)
    stockprice_btn.grid(row = 9,column = 1,pady = 10)
    cancle_btn = tk.Button(frame_l, text="取消委託", command = cancelOrder)
    cancle_btn.grid(row = 15,column = 2,padx = 10)


In [100]:
win = tk.Tk()#建立主視窗
win.title("股票回測")#視窗標題
win.geometry("1050x768")#寬*高
win.resizable(True, True)#限制縮放
#框架
frm = tk.Frame(win)
frm.grid(row = 0)
frm_l = tk.Frame(frm, ) #左視窗
frm_r = tk.Frame(frm,) #右視窗
frm_c = tk.Frame(frm,) #右下視窗
frm_k = tk.Frame(frm,) #左下視窗
frm_c2 = tk.Frame(frm,)
frm_k2 = tk.Frame(frm,)
frm_k3 = tk.Frame(frm,)
frm_k4 = tk.Frame(frm,)
frm_k5 = tk.Frame(frm,)
frm_k6 = tk.Frame(frm)
frm_k.grid(row = 2,column = 0)
frm_k2.grid(row = 3,column = 0)
frm_k3.grid(row = 4,column = 0)
frm_k4.grid(row = 5,column = 0)
frm_k5.grid(row = 0,column = 3)
frm_k6.grid(row = 1,column = 3)
frm_l.grid(row = 1,column = 0)
frm_r.grid(row = 1,column = 1)
frm_c.grid(row = 2,column = 1)
frm_c2.grid(row = 5,column = 1)
#----------Font--------------
ft1 = tkFont.Font(family='Fixdsys', size=16, weight=tkFont.BOLD)
ft2 = tkFont.Font(family='Fixdsys', size=14)
#Label(root, text='輸入區塊', font=ft1).grid()
#-----------------------Text---------------------------------
title = tk.Label(frm, text = "輸入區塊",font=ft1)
title.grid(row=0,column=0)
title2 = tk.Label(frm, text = "回測數據",font=ft1)
title2.grid(row=0,column=1)
title3 = tk.Label(frm_k, text = "選擇策略條件",font=ft1)
title3.grid(row=0,column=0,sticky='NW', pady = 5)
title4 = tk.Label(frm_k5, text = "虛擬交易/提醒功能",font=ft1)
title4.grid(row=0,column=0,sticky='NW', pady = 5)
stockCode = tk.Label(frm_l, text="輸入股票代碼:")#股票代碼
stockCode.grid(row=1,column=0,sticky='NW')
startTime = tk.Label(frm_l, text="輸入起始時間:")#股票日期
startTime.grid(row=2,column=0,sticky='NW')
endTime = tk.Label(frm_l, text='輸入結束時間')#股票日期
endTime.grid(row=3,column=0,sticky='NW')
start_money = tk.Label(frm_l, text="輸入初始金額:")#初始金額
start_money.grid(row=4,column=0,sticky='NW')
sProfit = tk.Label(frm_l, text="輸入停利條件:")#停利
sProfit.grid(row=5,column=0,sticky='NW')
sLoss = tk.Label(frm_l, text="輸入停損條件:")#停損
sLoss.grid(row=6,column=0,sticky='NW')

#-------------------------table------------------------------
table=ttk.Treeview(frm_r,show='headings')#表格
table["columns"]=("買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額","賣出獲利")
table.column("買/賣 狀態",width=100)   #表示列,不显示
table.column("交易日期",width=100)
table.column("股票價格",width=100)
table.column("股票庫存",width=100)
table.column("資金餘額",width=100)
table.column("賣出獲利",width=100)

table.heading("買/賣 狀態",text="買/賣 狀態")  #显示表头
table.heading("交易日期",text="交易日期")
table.heading("股票價格",text="股票價格")
table.heading("股票庫存",text="股票庫存")
table.heading("資金餘額",text="資金餘額")
table.heading("賣出獲利",text="賣出獲利")
table.grid()
#--------------------------textPanel-----------------------------------
#報酬率
pOfrate = tk.Label(frm_k4, text=' 報 酬 率 ',font=ft1)
pOfrate.grid(row =14, column = 0,sticky='NW', pady = 5)
#textPanel = tk.Text(frm_l,height=1, width=4)
#textPanel.grid(row =14,column=1,sticky='W')
#報酬率 1
ROP1 = tk.Label(frm_k4, text=' K值大小')
ROP1.grid(row =15, column = 0,sticky='NW')
textPanel1 = tk.Text(frm_k4,height=1, width=4)
textPanel1.grid(row =15, column = 1,sticky='NW')
ROP2 = tk.Label(frm_k4, text=' 連三反彈')
ROP2.grid(row =16, column = 0,sticky='NW')
textPanel2 = tk.Text(frm_k4,height=1, width=4)
textPanel2.grid(row =16, column = 1,sticky='NW')
ROP3 = tk.Label(frm_k4, text=' 黃金交叉')
ROP3.grid(row =17, column = 0,sticky='NW')
textPanel3 = tk.Text(frm_k4, height=1, width=4)
textPanel3.grid(row =17, column = 1,sticky='NW')

#預設輸入
stock = tk.StringVar(value='2330.TW')
time = tk.StringVar(value='2018')
timeEnd = tk.StringVar(value='2019')
sMoney =tk.StringVar(value='1000000')
pEntry = tk.StringVar(value='5')
lEntry = tk.StringVar(value='5')
#----------------------------entry-----------------------------------------
stockCode_entry = tk.Entry(frm_l,textvariable=stock)
stockCode_entry.grid(row=1,column=1,sticky='W')
startTime_entry = tk.Entry(frm_l,textvariable=time)
startTime_entry.grid(row=2,column=1,sticky='W')
endTime_entry = tk.Entry(frm_l,textvariable=timeEnd)
endTime_entry.grid(row=3,column=1,sticky='W')
start_money_entry = tk.Entry(frm_l,textvariable=sMoney)
start_money_entry.grid(row=4,column=1,sticky='W')
sProfit_entry = tk.Entry(frm_l,textvariable=pEntry)
sProfit_entry.grid(row=5,column=1,sticky='W')
sLoss_entry = tk.Entry(frm_l,textvariable=lEntry)
sLoss_entry.grid(row=6,column=1,sticky='W')
#-------------------------Optionmenu-------------------------#
optionList = ["K值小於設定值進場，大於設定值出場","K值連續3天小於設定值後反彈進場","黃金交叉與死亡交叉"]
v = tk.StringVar()
v.set("選擇條件")
optionmenu = tk.OptionMenu(frm_k2, v, *optionList)#command=func(*optionList))
optionmenu.grid(row=1,column=0,sticky='W')
#en.pack()

#-------------------------Button-------------------------#
btn = tk.Button(frm_k3, text="單一回測",command=callfunc)
btn.grid(row = 0,column = 0,pady = 5, padx = 5)
btn3 = tk.Button(frm_k6, text="虛擬交易平台",font=ft2, command=order)
btn3.grid(row = 0,column = 0, pady = 5)
btn4 = tk.Button(frm_k3, text="全部回測", command = Run3)
btn4.grid(row = 0,column = 1, pady = 5, padx = 5)
btn5 = tk.Button(frm_k6,text="指標自動提醒",font=ft2,command =callfunc2)
btn5.grid(row = 1,column = 0,pady =5)
btn6 = tk.Button(frm_k6,text="股價自動提醒",font=ft2,command =autoRemindPrice.priceRemind)
btn6.grid(row = 2,column = 0,pady =5)




win.mainloop()

2018-01-02 00:00:00
0
232.5
尚未達到買點
2018-01-03 00:00:00
1
237.0
尚未達到買點
2018-01-04 00:00:00
2
239.5
尚未達到買點
2018-01-05 00:00:00
3
240.0
尚未達到買點
2018-01-08 00:00:00
4
242.0
尚未達到買點
2018-01-09 00:00:00
5
242.0
尚未達到買點
2018-01-10 00:00:00
6
236.5
尚未達到買點
2018-01-11 00:00:00
7
235.0
尚未達到買點
2018-01-12 00:00:00
8
237.0
尚未達到買點
2018-01-15 00:00:00
9
240.0
尚未達到買點
2018-01-16 00:00:00
10
240.5
尚未達到買點
2018-01-17 00:00:00
11
242.0
尚未達到買點
2018-01-18 00:00:00
12
248.5
尚未達到買點
2018-01-19 00:00:00
13
255.5
尚未達到買點
2018-01-22 00:00:00
14
261.5
尚未達到買點
2018-01-23 00:00:00
15
266.0
尚未達到買點
2018-01-24 00:00:00
16
258.0
尚未達到買點
2018-01-25 00:00:00
17
258.0
尚未達到買點
2018-01-26 00:00:00
18
255.0
尚未達到買點
2018-01-29 00:00:00
19
258.5
尚未達到買點
2018-01-30 00:00:00
20
253.0
尚未達到買點
2018-01-31 00:00:00
21
255.0
尚未達到買點
2018-02-01 00:00:00
22
259.5
尚未達到買點
2018-02-02 00:00:00
23
259.5
尚未達到買點
2018-02-05 00:00:00
24
253.0
尚未達到買點
2018-02-06 00:00:00
25
239.0
尚未達到買點
2018-02-07 00:00:00
26
240.0
K值為: 19.00112233445567
買了一張股票，股價為: 240.0
[240

292
245.5
尚未達到買點
2019-03-22 00:00:00
293
248.5
尚未達到買點
2019-03-25 00:00:00
294
241.5
尚未達到買點
2019-03-26 00:00:00
295
244.0
尚未達到買點
2019-03-27 00:00:00
296
241.5
尚未達到買點
2019-03-28 00:00:00
297
242.0
尚未達到買點
2019-03-29 00:00:00
298
245.5
尚未達到買點
2019-04-01 00:00:00
299
245.5
尚未達到買點
2019-04-02 00:00:00
300
246.0
尚未達到買點
2019-04-03 00:00:00
301
246.5
尚未達到買點
2019-04-08 00:00:00
302
253.0
尚未達到買點
2019-04-09 00:00:00
303
254.0
尚未達到買點
2019-04-10 00:00:00
304
254.0
尚未達到買點
2019-04-11 00:00:00
305
252.0
尚未達到買點
2019-04-12 00:00:00
306
252.0
尚未達到買點
2019-04-15 00:00:00
307
255.5
尚未達到買點
2019-04-16 00:00:00
308
257.0
尚未達到買點
2019-04-17 00:00:00
309
261.5
尚未達到買點
2019-04-18 00:00:00
310
264.5
尚未達到買點
2019-04-19 00:00:00
311
264.5
尚未達到買點
2019-04-22 00:00:00
312
266.0
尚未達到買點
2019-04-23 00:00:00
313
268.0
尚未達到買點
2019-04-24 00:00:00
314
269.0
尚未達到買點
2019-04-25 00:00:00
315
267.5
尚未達到買點
2019-04-26 00:00:00
316
260.0
尚未達到買點
2019-04-29 00:00:00
317
259.5
尚未達到買點
2019-04-30 00:00:00
318
259.0
K值為: 19.678160919540204
買了一張

2019-12-03 00:00:00
465
307.0
K值為: 18.78306878306881
買了一張股票，股價為: 307.0
[307.5, 307.0]
剩餘資金為: 486500.0
擁有2張股票
2019-12-04 00:00:00
466
306.0
尚未達到買賣點
2019-12-05 00:00:00
467
312.0
現在的股價= 312.0
條件: 現股>=最後一次購買股價+10 = 
最後一次購買股價 307.0
最後一次購買股價+5 312.0
已達停利標準
賣出2張股票
股價為: 312.0
擁有資金: 1110500.0
獲利: 110500.0
報酬率: 11.05
2019-12-06 00:00:00
468
313.0
尚未達到買點
2019-12-09 00:00:00
469
316.0
尚未達到買點
2019-12-10 00:00:00
470
313.5
尚未達到買點
2019-12-11 00:00:00
471
316.5
尚未達到買點


總報酬率 11.05
2015-01-05 00:00:00
0
139.5
尚未達到買點
2015-01-06 00:00:00
1
133.5
尚未達到買點
2015-01-07 00:00:00
2
134.0
尚未達到買點
2015-01-08 00:00:00
3
138.0
尚未達到買點
2015-01-09 00:00:00
4
134.0
尚未達到買點
2015-01-12 00:00:00
5
132.0
尚未達到買點
2015-01-13 00:00:00
6
132.5
尚未達到買點
2015-01-14 00:00:00
7
130.0
尚未達到買點
2015-01-15 00:00:00
8
131.5
尚未達到買點
2015-01-16 00:00:00
9
137.0
尚未達到買點
2015-01-19 00:00:00
10
139.0
尚未達到買點
2015-01-20 00:00:00
11
138.0
尚未達到買點
2015-01-21 00:00:00
12
141.0
尚未達到買點
2015-01-22 00:00:00
13
140.0
尚未達到買點
2015-01-23 00:00:00
14
145.0
尚未達到買點

2015-08-17 00:00:00
149
124.0
K值為: 11.8729096989966
買了一張股票，股價為: 124.0
[126.0, 124.0]
剩餘資金為: 782000.0
擁有2張股票
2015-08-18 00:00:00
150
126.0
K值為: 13.15496098104788
買了一張股票，股價為: 126.0
[126.0, 124.0, 126.0]
剩餘資金為: 656000.0
擁有3張股票
2015-08-19 00:00:00
151
125.0
K值為: 16.60561660561655
買了一張股票，股價為: 125.0
[126.0, 124.0, 126.0, 125.0]
剩餘資金為: 531000.0
擁有4張股票
2015-08-20 00:00:00
152
126.0
尚未達到買賣點
2015-08-21 00:00:00
153
121.0
K值為: 15.873015873015817
買了一張股票，股價為: 121.0
[126.0, 124.0, 126.0, 125.0, 121.0]
剩餘資金為: 410000.0
擁有5張股票
2015-08-24 00:00:00
154
115.0
現在的股價= 115.0
條件: 現股<=最後一次購買股價-5 = 
最後一次購買股價 121.0
最後一次購買股價-5 116.0
已達停損標準
賣出5張股票
股價為: 115.0
i: 154
擁有資金: 985000.0
獲利: -15000.0
報酬率: -1.5
2015-08-25 00:00:00
155
123.5
尚未達到買點
2015-08-26 00:00:00
156
122.5
尚未達到買點
2015-08-27 00:00:00
157
124.0
尚未達到買點
2015-08-28 00:00:00
158
125.0
尚未達到買點
2015-08-31 00:00:00
159
129.0
尚未達到買點
2015-09-01 00:00:00
160
124.0
尚未達到買點
2015-09-02 00:00:00
161
125.5
尚未達到買點
2015-09-03 00:00:00
162
127.0
尚未達到買點
2015-09-04 00:00:00
1

尚未達到買點
2016-07-07 00:00:00
366
164.5
尚未達到買點
2016-07-11 00:00:00
367
170.0
尚未達到買點
2016-07-12 00:00:00
368
171.0
尚未達到買點
2016-07-13 00:00:00
369
168.5
尚未達到買點
2016-07-14 00:00:00
370
169.0
尚未達到買點
2016-07-15 00:00:00
371
169.5
尚未達到買點
2016-07-18 00:00:00
372
169.5
尚未達到買點
2016-07-19 00:00:00
373
170.5
尚未達到買點
2016-07-20 00:00:00
374
170.0
尚未達到買點
2016-07-21 00:00:00
375
172.0
尚未達到買點
2016-07-22 00:00:00
376
171.0
尚未達到買點
2016-07-25 00:00:00
377
171.5
尚未達到買點
2016-07-26 00:00:00
378
173.0
尚未達到買點
2016-07-27 00:00:00
379
175.0
尚未達到買點
2016-07-28 00:00:00
380
176.5
尚未達到買點
2016-07-29 00:00:00
381
172.5
尚未達到買點
2016-08-01 00:00:00
382
177.0
尚未達到買點
2016-08-02 00:00:00
383
176.0
尚未達到買點
2016-08-03 00:00:00
384
172.5
尚未達到買點
2016-08-04 00:00:00
385
174.5
尚未達到買點
2016-08-05 00:00:00
386
177.0
尚未達到買點
2016-08-08 00:00:00
387
178.0
尚未達到買點
2016-08-09 00:00:00
388
178.5
尚未達到買點
2016-08-10 00:00:00
389
179.5
尚未達到買點
2016-08-11 00:00:00
390
176.5
尚未達到買點
2016-08-12 00:00:00
391
177.5
尚未達到買點
2016-08-15 00:00:00
392
179.0
尚

...

2018-10-02 00:00:00
914
257.5
尚未達到買點
2018-10-03 00:00:00
915
260.0
尚未達到買點
2018-10-04 00:00:00
916
254.0
K值為: 12.96296296296282
買了一張股票，股價為: 254.0
[254.0]
剩餘資金為: 1057000.0
擁有1張股票
2018-10-05 00:00:00
917
250.0
K值為: 13.968253968253824
買了一張股票，股價為: 250.0
[254.0, 250.0]
剩餘資金為: 807000.0
擁有2張股票
2018-10-08 00:00:00
918
243.5
現在的股價= 243.5
條件: 現股<=最後一次購買股價-5 = 
最後一次購買股價 250.0
最後一次購買股價-5 245.0
已達停損標準
賣出2張股票
股價為: 243.5
i: 918
擁有資金: 1294000.0
獲利: 294000.0
報酬率: 29.4
2018-10-09 00:00:00
919
244.0
K值為: 10.190476190476046
買了一張股票，股價為: 244.0
[244.0]
剩餘資金為: 1050000.0
擁有1張股票
2018-10-11 00:00:00
920
227.5
現在的股價= 227.5
條件: 現股<=最後一次購買股價-5 = 
最後一次購買股價 244.0
最後一次購買股價-5 239.0
已達停損標準
賣出1張股票
股價為: 227.5
i: 920
擁有資金: 1277500.0
獲利: 277500.0
報酬率: 27.750000000000004
2018-10-12 00:00:00
921
237.0
K值為: 13.436359436359291
買了一張股票，股價為: 237.0
[237.0]
剩餘資金為: 1040500.0
擁有1張股票
2018-10-15 00:00:00
922
230.5
現在的股價= 230.5
條件: 現股<=最後一次購買股價-5 = 
最後一次購買股價 237.0
最後一次購買股價-5 232.0
已達停損標準
賣出1張股票
股價為: 230.5
i: 922
擁有資金: 1271000.0
獲利: 271000

K值為: 3.7037037037035248
買了一張股票，股價為: 238.0
[238.0]
剩餘資金為: 1084500.0
擁有1張股票
2019-05-21 00:00:00
1064
234.0
K值為: 1.851851851851673
買了一張股票，股價為: 234.0
[238.0, 234.0]
剩餘資金為: 850500.0
擁有2張股票
2019-05-22 00:00:00
1065
238.0
K值為: 8.770090845562365
買了一張股票，股價為: 238.0
[238.0, 234.0, 238.0]
剩餘資金為: 612500.0
擁有3張股票
2019-05-23 00:00:00
1066
230.0
現在的股價= 230.0
條件: 現股<=最後一次購買股價-5 = 
最後一次購買股價 238.0
最後一次購買股價-5 233.0
已達停損標準
賣出3張股票
股價為: 230.0
i: 1066
擁有資金: 1302500.0
獲利: 302500.0
報酬率: 30.25
2019-05-24 00:00:00
1067
233.0
K值為: 11.463693539165059
買了一張股票，股價為: 233.0
[233.0]
剩餘資金為: 1069500.0
擁有1張股票
2019-05-27 00:00:00
1068
231.0
K值為: 6.060606060605884
買了一張股票，股價為: 231.0
[233.0, 231.0]
剩餘資金為: 838500.0
擁有2張股票
2019-05-28 00:00:00
1069
230.5
K值為: 6.915306915306739
買了一張股票，股價為: 230.5
[233.0, 231.0, 230.5]
剩餘資金為: 608000.0
擁有3張股票
2019-05-29 00:00:00
1070
229.5
K值為: 6.157731157730981
買了一張股票，股價為: 229.5
[233.0, 231.0, 230.5, 229.5]
剩餘資金為: 378500.0
擁有4張股票
2019-05-30 00:00:00
1071
231.0
K值為: 12.975912975912799
買了一張股票，股價為: 231.0

2019-08-21 00:00:00
151
254.5
尚未達到買點
2019-08-22 00:00:00
152
254.0
尚未達到買點
2019-08-23 00:00:00
153
254.0
尚未達到買點
2019-08-26 00:00:00
154
248.5
尚未達到買點
2019-08-27 00:00:00
155
250.0
尚未達到買點
2019-08-28 00:00:00
156
252.0
尚未達到買點
2019-08-29 00:00:00
157
254.0
尚未達到買點
2019-08-30 00:00:00
158
259.0
尚未達到買點
2019-09-02 00:00:00
159
257.5
尚未達到買點
2019-09-03 00:00:00
160
254.0
尚未達到買點
2019-09-04 00:00:00
161
257.5
尚未達到買點
2019-09-05 00:00:00
162
263.0
尚未達到買點
2019-09-06 00:00:00
163
263.5
尚未達到買點
2019-09-10 00:00:00
164
261.5
尚未達到買點
2019-09-11 00:00:00
165
263.0
尚未達到買點
2019-09-12 00:00:00
166
262.5
尚未達到買點
2019-09-16 00:00:00
167
265.5
尚未達到買點
2019-09-17 00:00:00
168
265.0
尚未達到買點
2019-09-18 00:00:00
169
267.0
尚未達到買點
2019-09-19 00:00:00
170
265.0
尚未達到買點
2019-09-20 00:00:00
171
264.0
尚未達到買點
2019-09-23 00:00:00
172
264.0
尚未達到買點
2019-09-24 00:00:00
173
265.0
尚未達到買點
2019-09-25 00:00:00
174
266.0
尚未達到買點
2019-09-26 00:00:00
175
268.0
尚未達到買點
2019-09-27 00:00:00
176
272.0
尚未達到買點
2019-10-01 00:00:00
177
280.0
尚未達到買點
2

報酬率: 5.6000000000000005
2019-06-21 00:00:00
109
248.5
2019-06-24 00:00:00
110
241.0
2019-06-25 00:00:00
111
238.5
2019-06-26 00:00:00
112
234.5
2019-06-27 00:00:00
113
240.5
2019-06-28 00:00:00
114
239.0
2019-07-01 00:00:00
115
248.5
record: [239.0]
2019-06-28 00:00:00 K值為: 38.82915173237754 D值為: 40.412186379928286 2019-07-01 00:00:00 K值為: 61.63007765830346 D值為: 47.066656710473886
買了一張股票，股價為: 239.0
剩餘資金為: 817000.0
擁有1張股票
2019-07-02 00:00:00
116
249.0
2019-07-01 00:00:00 K值為: 61.63007765830346 D值為: 47.066656710473886 2019-07-02 00:00:00 K值為: 74.36155913978494 D值為: 58.27359617682195
手上有1張股票可賣
賣出1張股票
股價為: 248.5
擁有資金: 1065500.0
獲利: 65500.0
報酬率: 6.550000000000001
2019-07-03 00:00:00
117
242.5
2019-07-04 00:00:00
118
244.0
2019-07-05 00:00:00
119
243.0
2019-07-08 00:00:00
120
242.5
2019-07-09 00:00:00
121
242.0
2019-07-10 00:00:00
122
247.0
record: [242.0]
2019-07-09 00:00:00 K值為: 50.74404761904761 D值為: 55.10912698412694 2019-07-10 00:00:00 K值為: 56.99404761904761 D值為: 55.0099206349206
買了一張股票

312.0
2019-12-06 00:00:00
223
313.0
2019-12-05 00:00:00 K值為: 51.19047619047621 D值為: 32.363315696649 2019-12-06 00:00:00 K值為: 66.66666666666669 D值為: 48.32451499118162
手上有1張股票可賣
賣出1張股票
股價為: 312.0
擁有資金: 1142500.0
獲利: 142500.0
報酬率: 14.249999999999998
2019-12-09 00:00:00
224
316.0
2019-12-10 00:00:00
225
313.5
2019-12-11 00:00:00
226
316.0
